In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

Importação da base de dados

In [ ]:
# Carregar os dados
dados = pd.read_csv("dados.csv", sep=";")

# Tratar valores ausentes
dados['mecanismo_resistencia'].fillna('Desconhecido', inplace=True)
dados['mecanismo_resistencia'].replace('', 'Desconhecido', inplace=True)

# Filtrar para o estado CE
dados = dados[dados["state"] == "PR"]

In [ ]:
# Agrupar pelos campos necessários
summary = dados.groupby(['time', 'origin', 'classe_droga', 'mecanismo_resistencia']).size().reset_index(name='count')

# Criar os labels em ordem
labels = pd.concat([summary['time'], summary['origin'], summary['classe_droga'], summary['mecanismo_resistencia']]).unique()
label_to_index = {label: index for index, label in enumerate(labels)}

# Criar listas de índices para o gráfico de Sankey
time_index = summary['time'].map(label_to_index)
origin_index = summary['origin'].map(label_to_index)
classe_droga_index = summary['classe_droga'].map(label_to_index)
mecanismo_resistencia_index = summary['mecanismo_resistencia'].map(label_to_index)

In [ ]:
# Criar as listas de links (sources e targets)
source = []
target = []
value = []
link_colors = []

# Definir listas de cores
time_colors = [
    'rgba(205, 133, 63, 0.6)',   # Dark Orange
    'rgba(100, 149, 237, 0.6)',  # Cornflower Blue
    'rgba(60, 179, 113, 0.6)',   # Medium Sea Green
    'rgba(219, 112, 147, 0.6)',  # Pale Violet Red
    'rgba(189, 183, 107, 0.6)',  # Dark Khaki
]
origin_colors = [
    'rgba(255, 99, 71, 0.6)',  # Tomato
    'rgba(100, 149, 237, 0.6)',  # Cornflower Blue
]
classe_droga_colors = [
    'rgba(255, 99, 71, 0.6)',    # Tomato
    'rgba(60, 179, 113, 0.6)',   # Medium Sea Green
    'rgba(65, 105, 225, 0.6)',   # Royal Blue
    'rgba(255, 140, 0, 0.6)',    # Dark Orange
    'rgba(138, 43, 226, 0.6)',   # Blue Violet
    'rgba(255, 20, 147, 0.6)',   # Deep Pink
    'rgba(0, 191, 255, 0.6)',    # Deep Sky Blue
    'rgba(34, 139, 34, 0.6)',    # Forest Green
    'rgba(255, 215, 0, 0.6)',    # Gold
    'rgba(0, 128, 128, 0.6)',    # Teal
    'rgba(210, 105, 30, 0.6)',   # Chocolate
    'rgba(255, 69, 0, 0.6)',     # Orange Red
    'rgba(75, 0, 130, 0.6)',     # Indigo
    'rgba(124, 252, 0, 0.6)',    # Lawn Green
    'rgba(218, 112, 214, 0.6)',  # Orchid
    'rgba(244, 164, 96, 0.6)',   # Sandy Brown
    'rgba(0, 206, 209, 0.6)',    # Dark Turquoise
    'rgba(186, 85, 211, 0.6)',   # Medium Orchid
    'rgba(255, 228, 196, 0.6)',  # Bisque
    'rgba(128, 0, 0, 0.6)',      # Maroon
]

# Mapear estados para cores
time_to_color = {time: time_colors[i % len(time_colors)] for i, time in enumerate(summary['time'].unique())}
origin_to_color = {origin: origin_colors[i % len(origin_colors)] for i, origin in enumerate(summary['origin'].unique())}
classe_droga_to_color = {classe_droga: classe_droga_colors[i % len(classe_droga_colors)] for i, classe_droga in enumerate(summary['classe_droga'].unique())}

# Adicionar links de time para origin
for _, row in summary.iterrows():
    source.append(label_to_index[row['time']])
    target.append(label_to_index[row['origin']])
    value.append(row['count'])
    link_colors.append(time_to_color[row['time']])

# Adicionar links de origin para classe_droga
for _, row in summary.iterrows():
    source.append(label_to_index[row['origin']])
    target.append(label_to_index[row['classe_droga']])
    value.append(row['count'])
    link_colors.append(origin_to_color[row['origin']])

# Adicionar links de classe_droga para mecanismo_resistencia
for _, row in summary.iterrows():
    source.append(label_to_index[row['classe_droga']])
    target.append(label_to_index[row['mecanismo_resistencia']])
    value.append(row['count'])
    link_colors.append(classe_droga_to_color[row['classe_droga']])


Plotar o gráfico

In [ ]:
# Criar o gráfico de Sankey
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors
    )
))

fig.update_layout(title_text="Paraná: Período (mês) > Origem > Classe da Droga > Mecanismo de Resistência", font_size=16)

# Salvar a figura como um arquivo JPG
pio.write_image(fig, 'PR_sankey_plot.jpg', format='jpg', width=1920, height=1080, scale=2)

fig.show()